In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import asc, desc, sum as Fsum, udf

import datetime

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
spark = (
    SparkSession.builder.appName("Wrangling Data with Spark SQL")
    .config("spark.eventLog.gcMetrics.youngGenerationGarbageCollectors", "G1 Young Generation")
    .config("spark.eventLog.gcMetrics.oldGenerationGarbageCollectors", "G1 Old Generation, G1 Concurrent GC")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/28 10:45:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
path = "../data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [4]:
user_log.createOrReplaceTempView("user_log")

### Question 1
Which page did user id ""(empty string) NOT visit?

In [5]:
spark.sql("""
SELECT page 
FROM user_log 
WHERE userID = ''
""").drop_duplicates().sort("page").show()

+-----+
| page|
+-----+
|About|
| Help|
| Home|
|Login|
+-----+



### Question 2 - Reflect
Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

#### Upsides for using SQL
- Declarative, (sometimes) easier to follow and (probably) easier for spark to optimize

#### Upsides for using data frames
- Easier to compose and therefore easier to debug, reuse, and extend (esp. when doing exploration in an iterative fashion)

### Question 3

How many female users do we have in the data set?

In [6]:
spark.sql("""
SELECT gender, COUNT(gender)
FROM user_log
WHERE gender IS NOT NULL
GROUP BY gender
""").show()

+------+-------------+
|gender|count(gender)|
+------+-------------+
|     F|         3820|
|     M|         5844|
+------+-------------+



### Question 4

How many songs were played from the most played artist?

In [7]:
spark.sql("""
SELECT artist, COUNT(artist)
FROM user_log
WHERE artist IS NOT NULL
GROUP BY artist
ORDER BY COUNT(artist) DESC
LIMIT 1
""").show()

+--------+-------------+
|  artist|count(artist)|
+--------+-------------+
|Coldplay|           83|
+--------+-------------+



### Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [8]:
spark.sql("""
WITH 
    songs_per_session AS 
    (
        SELECT 
            count(song) as num_song
        FROM 
            user_log
        GROUP BY
            sessionId
    )
SELECT
    CAST(ROUND(AVG(num_song)) AS INT) AS avg_song_per_session
FROM
    songs_per_session
""").show()

+--------------------+
|avg_song_per_session|
+--------------------+
|                   6|
+--------------------+

